# Parsing the Json file - storing structured into PG admin and unstrcutred to mongodb (here: abstract)

Storing structured data into PGadmin - SQL - def functions  (use the final code for querying and analysis)

Code for data excluding abstract to store in sql

In [ ]:
import os
# Get full path to output_files
folder_path = os.path.abspath("Final_output_files")
print("Full path to JSON files:", folder_path)


Full path to JSON files: /Users/suhasinisingh/Desktop/Managing Data/Research/Final_output_files


In [ ]:
import json

with open("combined_nyt_articles.json") as f:
    nyt_data = json.load(f)

with open("combined_newsapifox_articles.json") as f:
    guardian_data = json.load(f)

with open("combined_guardian_articles.json") as f:
    newsapi_data_updated = json.load(f)


# SQL database

In [ ]:
import psycopg2

pg_conn = psycopg2.connect(
    host="localhost",
    database="news_articles",       # Your database name
    user="postgres",      # Your PostgreSQL username
    password="123",  # Your PostgreSQL password
    port="5432"
)

pg_cursor = pg_conn.cursor()


In [ ]:
# Creating 3 tables for 3 newsAPIs
pg_cursor = pg_conn.cursor()

# NYTimes
pg_cursor.execute("""
CREATE TABLE IF NOT EXISTS nytimes (
    id SERIAL PRIMARY KEY,
    title TEXT,
    abstract TEXT,
    url TEXT,
    published_at TIMESTAMP
);
""")

# Guardian
pg_cursor.execute("""
CREATE TABLE IF NOT EXISTS guardian (
    id SERIAL PRIMARY KEY,
    title TEXT,
    section TEXT,
    url TEXT,
    published_at TIMESTAMP
);
""")

# NewsAPI
pg_cursor.execute("""
CREATE TABLE IF NOT EXISTS newsapi (
    id SERIAL PRIMARY KEY,
    title TEXT,
    source TEXT,
    url TEXT,
    published_at TIMESTAMP
);
""")

pg_conn.commit()


In [ ]:
article.get("source")

'Guardian'

In [ ]:
# Inserting data in 3 tables
#nytimes
for article in nyt_data:
    pg_cursor.execute("""
        INSERT INTO nytimes (title, abstract, url, published_at)
        VALUES (%s, %s, %s, %s)
    """, (
        article.get("title"),
        article.get("abstract"),
        article.get("url"),
        article.get("published_date")
    ))
#Guardian
for article in guardian_data:
    pg_cursor.execute("""
        INSERT INTO guardian (title, section, url, published_at)
        VALUES (%s, %s, %s, %s)
    """, (
        article.get("webTitle"),
        article.get("sectionName"),
        article.get("webUrl"),
        article.get("webPublicationDate")
    ))
#newsAPI
for article in newsapi_data:
    pg_cursor.execute("""
        INSERT INTO newsapi (title, source, url, published_at)
        VALUES (%s, %s, %s, %s)
    """, (
        article.get("title"),
        article.get("source"),  # ✅ Use directly
        article.get("url"),
        article.get("publishedAt")
    ))

# MongodDb database

In [ ]:
#
import os
import json
from glob import glob
from pymongo import MongoClient

from pymongo import MongoClient
import json

# Connect to local MongoDB instance
client = MongoClient("mongodb://localhost:27017/")

# Select (or create) the database
db = client["news_articles"]

In [ ]:
import json

# NYTimes
with open("/Users/suhasinisingh/Desktop/Managing Data/Research/Final_output_files/combined_nyt_articles.json") as f:
    nyt_data = json.load(f)

# Guardian
with open("/Users/suhasinisingh/Desktop/Managing Data/Research/Final_output_files/combined_guardian_articles.json") as f:
    guardian_data = json.load(f)

# NewsAPI (Fox)
with open("/Users/suhasinisingh/Desktop/Managing Data/Research/Final_output_files/combined_newsapifox_articles.json") as f:
    newsapi_data_updated = json.load(f)


In [ ]:
# ✅ Insert into MongoDB collections
db.nytimes.insert_many(nyt_data)
db.guardian.insert_many(guardian_data)
db.newsapi_updated.insert_many(newsapi_data)

print(type(nyt_data), type(nyt_data[0]))


<class 'list'> <class 'dict'>


In [ ]:
print("🧾 Document Counts in MongoDB:")
print("NYTimes:", db.nytimes.count_documents({}))
print("Guardian:", db.guardian.count_documents({}))
print("NewsAPI:", db.newsapi.count_documents({}))


🧾 Document Counts in MongoDB:
NYTimes: 1090
Guardian: 4106
NewsAPI: 619


# def- functions() for research questions

In [ ]:
# checking the structure of SQL database
def describe_table(table_name):
    pg_cursor.execute(f"""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{table_name}'
        ORDER BY ordinal_position;
    """)
    columns = pg_cursor.fetchall()
    print(f"Structure of `{table_name}` table:")
    for col in columns:
        print(f" - {col[0]}: {col[1]}")
    print("\n")

def preview_table(table_name):
    pg_cursor.execute(f"SELECT * FROM {table_name} LIMIT 10;")
    rows = pg_cursor.fetchall()
    print(f"First 10 rows of `{table_name}`:")
    for row in rows:
        print(row)
    print("\n")

for table in ['nytimes', 'guardian', 'newsapi']:
    describe_table(table)
    preview_table(table)


Structure of `nytimes` table:
 - id: integer
 - title: text
 - abstract: text
 - url: text
 - published_at: timestamp without time zone


First 10 rows of `nytimes`:
(1, 'The Last 2 Months — and Next 2 Years — of U.S. Politics', 'Ezra Klein answers listener questions about the first two months of the second Trump term and the options Democrats and civil society have in response.', None, datetime.datetime(2025, 3, 28, 9, 4, 8))
(2, 'Trump’s Budget Chief Eyes Deep Cuts to ‘Woke and Weaponized’ Spending', 'Russell T. Vought, leader of the Office of Management and Budget, aims to marry extreme austerity with his Christian values.', None, datetime.datetime(2025, 3, 28, 9, 3))
(3, 'DoorDash Announces ‘Buy Now, Pay Later’ Partnership With Klarna', 'DoorDash said the arrangement with the financial tech service Klarna would allow customers to defer payments on orders such as takeout from restaurants, makeup and electronics.', None, datetime.datetime(2025, 3, 23, 20, 18, 47))
(4, 'Trump Cleaves 

In [ ]:
# checking the strcuture of Mongodb

print("Collections in 'news_articles' DB:")
print(db.list_collection_names())

def preview_collection(collection_name):
    print(f"\n📁 Collection: {collection_name}")
    cursor = db[collection_name].find().limit(10)
    for doc in cursor:
        print(doc)
for collection in ["nytimes", "guardian", "newsapi"]:
    preview_collection(collection)

def show_fields(collection_name):
    print(f"\n🔎 Fields in `{collection_name}`:")
    sample_doc = db[collection_name].find_one()
    if sample_doc:
        for key in sample_doc.keys():
            print(f" - {key}")
    else:
        print("No documents found.")

for col in ["nytimes", "guardian", "newsapi"]:
    show_fields(col)


Collections in 'news_articles' DB:
['nytimes', 'newsapi', 'guardian']

📁 Collection: nytimes
{'_id': ObjectId('67feaa0f12d6f2305a474563'), 'source': 'The New York Times', 'category': 'Education', 'abstract': 'Ezra Klein answers listener questions about the first two months of the second Trump term and the options Democrats and civil society have in response.', 'web_url': 'https://www.nytimes.com/2025/03/28/opinion/ezra-klein-podcast-ama.html', 'processed_date': '2025-04-11T04:56:14.664146', 'title': 'The Last 2 Months — and Next 2 Years — of U.S. Politics', 'author': 'By Ezra Klein', 'published_date': '2025-03-28T09:04:08+0000'}
{'_id': ObjectId('67feaa0f12d6f2305a474564'), 'source': 'The New York Times', 'category': 'Education', 'abstract': 'Russell T. Vought, leader of the Office of Management and Budget, aims to marry extreme austerity with his Christian values.', 'web_url': 'https://www.nytimes.com/2025/03/28/us/politics/trump-budget-cuts-russell-vought.html', 'processed_date': '20

In [ ]:
def print_keys(collection_name):
    doc = db[collection_name].find_one()
    if doc:
        print(f"\n🔑 Keys in `{collection_name}` collection:")
        for key in doc.keys():
            print(f" - {key}")
    else:
        print(f"\n⚠️ No documents found in `{collection_name}` collection.")

# Run for all three
for collection in ["nytimes", "guardian", "newsapi"]:
    print_keys(collection)



🔑 Keys in `nytimes` collection:
 - _id
 - source
 - category
 - abstract
 - web_url
 - processed_date
 - title
 - author
 - published_date

🔑 Keys in `guardian` collection:
 - _id
 - id
 - type
 - sectionId
 - sectionName
 - webPublicationDate
 - webTitle
 - webUrl
 - apiUrl
 - isHosted
 - pillarId
 - pillarName
 - issue
 - source

🔑 Keys in `newsapi` collection:
 - _id
 - source
 - author
 - title
 - description
 - url
 - urlToImage
 - publishedAt
 - content
 - issue


In [ ]:
# confirming the final issues under newsapi and guaridan
print("📂 Distinct issues in `guardian` collection:")
print(db.guardian.distinct("issue"))

print("\n📂 Distinct issues in `newsapi` collection:")
print(db.newsapi.distinct("issue"))


📂 Distinct issues in `guardian` collection:
['carbon emission', 'climate change', 'diversity education', 'education inequality', 'education policy', 'elections', 'foreign policy', 'green tech', 'immigration policy', 'partisan divide', 'recycling', 'remote learning', 'school funding', 'student loan debt', 'sustainability']

📂 Distinct issues in `newsapi` collection:
['air pollution', 'climate change', 'diversity education', 'education inequality', 'education policy', 'elections', 'foreign policy', 'green tech', 'immigration policy', 'partisan divide', 'recycling', 'remote learning', 'school funding', 'student loan debt', 'sustainability']


In [ ]:
# adding the issue column in Nytimes collection with the same labels
issue_keywords = {
    "carbon emission": ["carbon", "emission"],
    "climate change": ["climate change", "global warming", "climate crisis"],
    "diversity education": ["diversity", "inclusive education", "equity"],
    "education inequality": ["education gap", "inequality", "achievement gap"],
    "education policy": ["education policy", "curriculum", "reform"],
    "elections": ["election", "vote", "voting"],
    "foreign policy": ["foreign policy", "diplomacy", "international relations"],
    "green tech": ["green tech", "renewable energy", "solar", "wind"],
    "immigration policy": ["immigration", "border", "migrant"],
    "partisan divide": ["partisan", "polarization", "division"],
    "recycling": ["recycling", "waste", "plastic"],
    "remote learning": ["remote learning", "online school", "distance education"],
    "school funding": ["school funding", "education budget", "public schools"],
    "student loan debt": ["student loan", "college debt", "tuition"],
    "sustainability": ["sustainability", "sustainable", "eco-friendly"]
}
for issue, keywords in issue_keywords.items():
    for keyword in keywords:
        db.nytimes.update_many(
            {
                "$or": [
                    {"title": {"$regex": keyword, "$options": "i"}},
                    {"abstract": {"$regex": keyword, "$options": "i"}}
                ]
            },
            {"$set": {"issue": issue}}
        )


In [ ]:
#checking if it worked
print("Issues now in NYTimes collection:")
print(db.nytimes.distinct("issue"))


Issues now in NYTimes collection:
['carbon emission', 'climate change', 'diversity education', 'education inequality', 'education policy', 'elections', 'foreign policy', 'green tech', 'immigration policy', 'partisan divide', 'recycling', 'school funding', 'student loan debt', 'sustainability']


In [ ]:
def print_keys(collection_name):
    doc = db[collection_name].find_one()
    if doc:
        print(f"\n🔑 Keys in `{collection_name}` collection:")
        for key in doc.keys():
            print(f" - {key}")
    else:
        print(f"\n⚠️ No documents found in `{collection_name}` collection.")

# Run for all three
for collection in ["nytimes"]:
    print_keys(collection)


🔑 Keys in `nytimes` collection:
 - _id
 - source
 - category
 - abstract
 - web_url
 - processed_date
 - title
 - author
 - published_date


In [ ]:
print(db.nytimes.find_one({"issue": {"$exists": True}}))


{'_id': ObjectId('67feaa0f12d6f2305a474566'), 'source': 'The New York Times', 'category': 'Education', 'abstract': 'President Trump said Friday that he was tasking other government entities with core functions of the Education Department as he moves to dismantle an agency conservatives have long criticized.', 'web_url': 'https://www.nytimes.com/2025/03/21/us/politics/trump-education-department-student-loans.html', 'processed_date': '2025-04-11T04:56:14.664179', 'title': 'Trump Cleaves Student Loan and Special Education Services From Education Dept.', 'author': 'By Erica L. Green, Karoun Demirjian and Alan Blinder', 'published_date': '2025-03-21T17:18:42+0000', 'issue': 'student loan debt'}


In [ ]:
# First research question with fron end design
import streamlit as st
from pymongo import MongoClient
from datetime import datetime

# ---------- DATABASE CONNECTION ---------- #
def connect_to_mongodb():
    client = MongoClient("mongodb://localhost:27017/")
    return client["news_articles"]

# ---------- CATEGORY & ISSUE MAPPING ---------- #
category_issues = {
    "education": [
        "education inequality", "remote learning", "diversity education",
        "school funding", "student loan debt"
    ],
    "politics": [
        "foreign policy", "partisan divide", "elections",
        "immigration policy", "education policy"
    ],
    "environment": [
        "carbon emission", "climate change", "green tech",
        "recycling", "remote learning", "sustainability"
    ]
}

# ---------- HELPER FUNCTIONS ---------- #
def fetch_articles(db, collection_name, issue):
    collection = db[collection_name]
    return list(collection.find({"issue": issue}))

def display_articles(collection_name, articles):
    st.markdown(f"### 📰 {collection_name.upper()} News", unsafe_allow_html=True)
    st.markdown(
        f"<div style='background-color:beige; padding:10px; border-radius:10px;'>",
        unsafe_allow_html=True
    )
    if articles:
        st.write(f"**Total Articles Pulled:** {len(articles)}")
        for article in articles[:1]:
            st.write(f"**Author:** {article.get('author', 'N/A')}")
            st.write(f"**Title:** {article.get('title', 'No Title')}")
            st.write(f"**Published Date:** {format_date(article.get('published_date'))}")
            st.write(f"**Abstract:** {article.get('abstract', 'No abstract')}")
    else:
        st.warning("No articles found.")
    st.markdown("</div>", unsafe_allow_html=True)

# ---------- UTILITIES ---------- #
def format_date(date_str):
    if not date_str:
        return "N/A"
    try:
        dt = datetime.fromisoformat(date_str.replace("Z", "+00:00"))
        return dt.strftime("%Y-%m-%d %H:%M")
    except Exception:
        return date_str

# ---------- MAIN APP ---------- #
def run_news_dashboard():
    db = connect_to_mongodb()

    st.image("https://cdn.pixabay.com/photo/2017/09/08/04/47/news-2726331_1280.jpg", use_column_width=True)
    st.title("🗞️ News Dashboard - MongoDB Integration")

    category = st.selectbox("Select a Category:", list(category_issues.keys()))
    issue = st.selectbox("Select an Issue:", category_issues[category])

    col1, col2, col3 = st.columns(3)

    with col1:
        display_articles("nytimes", fetch_articles(db, "nytimes", issue))

    with col2:
        display_articles("newsapi", fetch_articles(db, "newsapi", issue))

    with col3:
        display_articles("guardian", fetch_articles(db, "guardian", issue))

# ---------- STREAMLIT ENTRY POINT ---------- #
if __name__ == '__main__':
    run_news_dashboard()


2025-04-15 15:19:25.707 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
# checking the file locations of json and code file
import os

# Get the absolute path of the current Python script
script_dir = os.path.abspath(os.getcwd())
print("📄 Current script is running from:\n", script_dir)

# JSON filenames
json_files = [
    "combined_nyt_articles.json",
    "combined_guardian_articles.json",
    "combined_newsapifox_articles.json"
]

print("\n📁 Full paths to each JSON file:")

# Print full path for each file
for file in json_files:
    full_path = os.path.join(script_dir, file)
    print(f"- {file}: {full_path}")


📄 Current script is running from:
 /Users/suhasinisingh/Desktop/Managing Data/Research

📁 Full paths to each JSON file:
- combined_nyt_articles.json: /Users/suhasinisingh/Desktop/Managing Data/Research/combined_nyt_articles.json
- combined_guardian_articles.json: /Users/suhasinisingh/Desktop/Managing Data/Research/combined_guardian_articles.json
- combined_newsapifox_articles.json: /Users/suhasinisingh/Desktop/Managing Data/Research/combined_newsapifox_articles.json


In [ ]:
def print_first_five_rows(table_name):
    pg_cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
    rows = pg_cursor.fetchall()

    print(f"🔍 First 5 rows from `{table_name}`:")
    for i, row in enumerate(rows, start=1):
        print(f"{i}. {row}")
    print("\n")

# Loop through each table
for table in ['nytimes', 'guardian', 'newsapi']:
    print_first_five_rows(table)


🔍 First 5 rows from `nytimes`:
1. (1, 'The Last 2 Months — and Next 2 Years — of U.S. Politics', 'Ezra Klein answers listener questions about the first two months of the second Trump term and the options Democrats and civil society have in response.', None, datetime.datetime(2025, 3, 28, 9, 4, 8))
2. (2, 'Trump’s Budget Chief Eyes Deep Cuts to ‘Woke and Weaponized’ Spending', 'Russell T. Vought, leader of the Office of Management and Budget, aims to marry extreme austerity with his Christian values.', None, datetime.datetime(2025, 3, 28, 9, 3))
3. (3, 'DoorDash Announces ‘Buy Now, Pay Later’ Partnership With Klarna', 'DoorDash said the arrangement with the financial tech service Klarna would allow customers to defer payments on orders such as takeout from restaurants, makeup and electronics.', None, datetime.datetime(2025, 3, 23, 20, 18, 47))
4. (4, 'Trump Cleaves Student Loan and Special Education Services From Education Dept.', 'President Trump said Friday that he was tasking other g